In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

In [3]:
# Set patient and record (same channels)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]

# observations = ["chb02_16", "chb02_19"]

# observations = ["chb04_28"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]

# observations = ["PN14_4"]
# observations = ["chb01_16"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache data

In [5]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

Removed file: .\data\cache\datasets\GCS-7461d1da0d4c1403016b336840abe8b6
Removed folder: .\data\cache\oracles\GCS-7461d1da0d4c1403016b336840abe8b6


## Run Gretel experiment

In [6]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [7]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
print('--------------------------------------')

for i in range(len(eval_manager._evaluators)):
    # context.logger.info(f"graph_ids: {eval_manager._evaluators[i].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
    context.logger.info(f"graph_metrics: {eval_manager._evaluators[i].get_instance_and_counterfactual_graph_metrics()}")

['chb01_03', 'chb01_04', 'chb01_15', 'chb01_16', 'chb01_18', 'chb01_21', 'chb01_26']
Generating context for: config/GCS-GCN.jsonc
2025-05-05 14:49:05,694 | INFO | 8532 - Executing: config/GCS-GCN.jsonc Run: -1
2025-05-05 14:49:05,726 | INFO | 8532 - Creating the evaluation manager.......................................................
2025-05-05 14:49:05,741 | INFO | 8532 - Creating the PAIRED evaluators...............................................................
2025-05-05 14:49:05,813 | INFO | 8532 - Creating: GCS-7461d1da0d4c1403016b336840abe8b6
2025-05-05 14:49:05,829 | INFO | 8532 - Instantiating: src.dataset.generators.gcs.GCS
2025-05-05 14:49:13,627 | INFO | 8532 - Saved: GCS-7461d1da0d4c1403016b336840abe8b6
2025-05-05 14:49:13,689 | INFO | 8532 - Created: GCS-7461d1da0d4c1403016b336840abe8b6
2025-05-05 14:49:13,737 | INFO | 8532 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-05-05 14:49:13,743 | INFO | 8532 - Instantiating: torch.optim.RMSprop
2025-05-05 14:49:13,743 

c:\Users\zanno\explainability\GRETEL-repo\src\explainer\generative\gans\graph\res_gen.py:64: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  edge_weights = torch.add(edge_weights, rebuild_adj_matrix(len(node_features), edge_list, edge_attr.T, self.device))
c:\Users\zanno\explainability\GRETEL-repo\src\explainer\generative\gans\graph\model.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  D_loss = torch.mean(self.loss_fn(y_pred.squeeze().double(), y_batch.double())

2025-05-05 15:35:49,362 | INFO | 8532 - Epoch 1	 Loss_D =  0.6930	 Loss_G =  0.6965
2025-05-05 15:35:49,481 | INFO | 8532 - Epoch 2	 Loss_D =  0.6930	 Loss_G =  0.6954
2025-05-05 15:35:49,587 | INFO | 8532 - Epoch 3	 Loss_D =  0.6924	 Loss_G =  0.6953
2025-05-05 15:35:49,695 | INFO | 8532 - Epoch 4	 Loss_D =  0.6921	 Loss_G =  0.6947
2025-05-05 15:35:49,803 | INFO | 8532 - Epoch 5	 Loss_D =  0.6917	 Loss_G =  0.6968
2025-05-05 15:35:49,904 | INFO | 8532 - Epoch 6	 Loss_D =  0.6970	 Loss_G =  0.6946
2025-05-05 15:35:50,016 | INFO | 8532 - Epoch 7	 Loss_D =  0.6965	 Loss_G =  0.6953
2025-05-05 15:35:50,111 | INFO | 8532 - Epoch 8	 Loss_D =  0.6924	 Loss_G =  0.6946
2025-05-05 15:35:50,220 | INFO | 8532 - Epoch 9	 Loss_D =  0.6935	 Loss_G =  0.6946
2025-05-05 15:35:50,321 | INFO | 8532 - Epoch 10	 Loss_D =  0.6943	 Loss_G =  0.6935
2025-05-05 15:35:50,420 | INFO | 8532 - Epoch 11	 Loss_D =  0.6935	 Loss_G =  0.6938
2025-05-05 15:35:50,520 | INFO | 8532 - Epoch 12	 Loss_D =  0.6945	 Loss_G

In [8]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [9]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 05/05/2025, ore 22:04


In [10]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [11]:
# from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

# results_path = "output\\results"
# stats_file_path = "output"
# res = data_analyzer.create_aggregated_dataframe(results_path)
# res.to_csv(stats_file_path)
# res